# Importando Bibliotecas

In [1]:
pip install librosa

Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch
import librosa
import logging
import numpy as np

# Importando o dataset
Aqui é importante ressaltar que temos que adicionar o dataset que já foi atualizado e gerado no notebook *1_Model_Noise_Detection_WER.*

In [4]:
import pandas as pd

# Substitua com o caminho para o seu arquivo CSV
csv_file = "\dataset_atualizado.csv"

# Carregar o dataset
dataset = pd.read_csv(csv_file)


# Calculando o SNRs
Vamos calcular o SNRs dos áudios através dos áudios com ruído e sem ruído, para termos um bom resultado. 

"Signal-to-Noise Ratios" (Relações Sinal-Ruído), o SNRs indicam a clareza ou qualidade do sinal, onde uma SNR mais alta significa que o sinal é mais forte em comparação com o ruído de fundo, resultando em melhor qualidade de áudio ou dados.

Com base no nível de SNRs, classificamos os áudios, para análises futuras.









In [6]:
import os
import pandas as pd
import librosa
import numpy as np

# Função para calcular a relação sinal-ruído (SNR) entre dois arquivos de áudio
def calculate_snr(reference_path, comparison_path):
    # Carrega o sinal de referência e seu sample rate
    reference_signal, sr_reference = librosa.load(reference_path, sr=None)
    # Carrega o sinal de comparação e seu sample rate
    comparison_signal, sr_comparison = librosa.load(comparison_path, sr=None)
    
    # Resample o sinal de comparação se os sample rates forem diferentes
    if sr_reference != sr_comparison:
        comparison_signal = librosa.resample(comparison_signal, orig_sr=sr_comparison, target_sr=sr_reference)
    
    # Calcula a diferença entre o sinal de referência e o sinal de comparação
    difference = reference_signal - comparison_signal
    # Calcula a potência do sinal de referência
    reference_power = np.mean(reference_signal ** 2)
    # Calcula a potência da diferença (ruído)
    difference_power = np.mean(difference ** 2)
    
    # Verifica se os valores são válidos e evita divisão por zero
    if reference_power > 0 and difference_power > 0:
        # Calcula o SNR em decibéis
        snr = 10 * np.log10(reference_power / difference_power)
    else:
        # Define SNR como NaN se os valores não forem válidos
        snr = float('nan')  # ou outro valor que faça sentido no seu contexto, como 0
    
    return snr

# Função para classificar o nível de ruído com base no SNR
def classify_noise_level(snr):
    if snr >= 29:
        return "Nenhum"
    elif 20 < snr <= 28:
        return "Pouco"
    elif 11 < snr <= 20:
        return "Médio"
    else:
        return "Muito"

# Adiciona as novas colunas ao DataFrame
dataset['SNRs'] = None
dataset['nível_de_ruído_SNRs'] = None

# Caminhos das pastas de arquivos originais e denoised, no original_audio_dir colocamos o áudio com os ruídos
# No denoised_audio_dir colocamos os áudios sem ruídos
original_audio_dir = '\original_audios'  # Substitua pelo caminho correto
denoised_audio_dir = '\denoised_audios'  # Substitua pelo caminho correto

# Processando cada arquivo de áudio no DataFrame
for index, row in dataset.iterrows():
    # Obtém o ID do áudio segmentado da linha atual
    id_audio = row['audio_segmentado']
    # Constroi o caminho completo do arquivo de áudio original
    original_path = os.path.join(original_audio_dir, id_audio)
    # Constroi o caminho completo do arquivo de áudio denoised
    denoised_path = os.path.join(denoised_audio_dir, id_audio)

    # Verifica se os arquivos de áudio existem
    if os.path.exists(original_path) and os.path.exists(denoised_path):
        # Calcula o SNR entre o áudio original e o denoised
        snr_value = calculate_snr(original_path, denoised_path)
        # Classifica o nível de ruído com base no SNR
        noise_level = classify_noise_level(snr_value)
        # Atualiza o DataFrame com os valores calculados
        dataset.at[index, 'SNRs'] = snr_value
        dataset.at[index, 'nível_de_ruído_SNRs'] = noise_level
    else:
        # Imprime uma mensagem se um ou ambos os arquivos não forem encontrados
        print(f"Arquivo(s) não encontrado(s) para: {id_audio}")

# Salva o DataFrame atualizado em um arquivo CSV
dataset.to_csv('\dataset_atualizados_snrs.csv', index=False)  # Substitua pelo caminho desejado para o arquivo CSV

print("Finalizada a análise de SNR e nível de ruído.")


Finalizada a análise de SNR e nível de ruído.


Ao final da execução ele vai gerar um novo dataset, assim estaremos prontos para implementar a nossa análise, no próximo notebook.